In [1]:
import json
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re
from requests.compat import urlparse, urljoin
import sqlite3
from requests import Session
from datetime import datetime, timedelta 

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params = params if method.upper() == 'GET' else '',
                               data = params if method.upper() == 'POST' else '',
                               headers = headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60) # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [ ]:
conn = sqlite3.connect('news4.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS head;
    CREATE TABLE head(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        head TEXT NOT NULL,
        cdate TEXT NOT NULL,
        wdate TEXT NOT NULL,
        ref INTEGER NOT NULL,
        page INTEGER NOT NULL
    );
''')

cur.executescript('''
    DROP TABLE IF EXISTS history;
    CREATE TABLE history(
        pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        seen TEXT NOT NULL,
        ref INTEGER NOT NULL
    );
''')

In [ ]:
seen = list()
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
          'Cookie': 'dable_uid=56241276.1588668566321; __gads=ID=32ec6ccd891b6cb1:T=1588668565:S=ALNI_Mb-cg_Eg1b8md7cdvKumpaG3T20ww; gtmdlkr=; _gid=GA1.2.295727024.1597748733; onlytitle=check; _gat_UA-109163096-1=1; _gat_UA-109163096-3=1; _gat_UA-136144676-1=1; _ga_PBVPGVW46M=GS1.1.1597748731.3.1.1597749317.0; _ga=GA1.2.402869033.1588668573'
          }
params = {
    'page': '1'
}


url = 'https://www.hankyung.com/all-news/?page=1'

urls = list()
urls.append(url)
count = 0
    
while urls:
    try:
        count += 1
        seed = urls.pop(0)
        
        params['page'] = re.search('page=(\d+)', urlparse(seed).query).group(1)
        resp = download(seed, params=params,headers=headers, method='GET')
        dom = BeautifulSoup(resp.text, 'html.parser')
        
        seen.append(seed)
        cur.execute('''
            INSERT INTO history(seen, ref) VALUES(?, 307)
        ''', [seed])
        
        for _ in [_['href'] for _ in dom.select('#container > div.contents_wrap > div.contents > div.article_content > div.paging > a')[2:-1]]:
            newUrls = urljoin(seed, _)
            if newUrls not in urls and newUrls not in seen:
                urls.append(newUrls)
        
        if dom.select('#container > div.contents_wrap > div.contents > div.article_content > div.daily_article > div > ul > li > div > h3 > a') != None:
            head = [_.text.strip() for _ in dom.select('#container > div.contents_wrap > div.contents > div.article_content > div.daily_article > div > ul > li > div > h3 > a')]
            date = [re.search('(\d{8})', _['href']).group(1) for _ in dom.select('#container > div.contents_wrap > div.contents > div.article_content > div.daily_article > div > ul > li > div > h3 > a')]
            time = [_.text.strip() for _ in dom.select('#container > div.contents_wrap > div.contents > div.article_content > div.daily_article > div > ul > li > div.txt > p.time')]
            t = list()
            for i in range(len(date)):
                t.append(date[i]+' '+time[i])
            cdate = str(datetime.now()).split('.')[0]
            page = re.search('page=(\d+)', urlparse(seed).query).group(1)
            if len(head)==len(t):
                for _ in range(0,len(head)):
                    cur.execute('''
                        INSERT INTO head(head, cdate, wdate, page, ref) VALUES(?,?,?,?,307)
                        ''', [head[_], t[_], cdate, page])
                    conn.commit()

        if count%100 == 0:
                print(count)
     
    except Exception as e:
            print('Error',e)